<a href="https://colab.research.google.com/github/Cerabbite/Easy-conalng/blob/main/Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For each word in the top 10.000 most used English, Spanish, Japanese and Chinese words. There will be 100-1.000 sample sentences with those words (written in using the english alphabet). Using these sentences this script is going to work out how related each word is to another word.

In [1]:
# Import libraries
import tensorflow